In [ ]:
from sctoolbox.utils.jupyter import bgcolor, _compare_version

# change the background of input cells
bgcolor("PowderBlue", select=[2, 4, 6, 8, 10, 13, 15])

nb_name = "0A1_ligand_receptor.ipynb"

_compare_version(nb_name)

# 0A - Receptor-Ligand Analysis
<hr style="border:2px solid black"> </hr>

## 1 - Description

**Requires an anndata object with precomputed clusters. Clusters can be generated with the clustering notebook (`rna_analysis/notebooks/04_clustering.ipynb`).**

Receptor-ligand interactions play a crucial role in mediating cellular communication and signaling processes in biological systems. In the context of single-cell data analysis, studying receptor-ligand interactions provides insights into cell-cell communication networks and their impact on various cellular functions and behaviors.  
This notebook provides the tools to investigate and visualize the ligand-receptor interactions between conditions and/or clusters.

--------------

## 2 - Setup

In [ ]:
import sctoolbox.tools.receptor_ligand as rl
import sctoolbox.utils as utils
import pandas as pd
from sctoolbox import settings
settings.settings_from_config("config.yaml", key="0A1")

with pd.option_context("display.max.rows", None, "display.max_colwidth", None):
    display(utils.general.get_version_report(report="versions.yml"))

--------------

## 3 - Load adata

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
anndata_file = "anndata_4.h5ad"

___

In [ ]:
adata = utils.adata.load_h5ad(anndata_file)

with pd.option_context("display.max.rows", 5, "display.max.columns", None):
    display(adata)
    display(adata.obs)
    display(adata.var)

--------------

## 4 - Download receptor-ligand database

Supply a receptor-ligand database. This can be done in two ways:

1. Provide the name of a recognized [LIANA resource](https://liana-py.readthedocs.io/en/latest/notebooks/prior_knowledge.html#Ligand-Receptor-Interactions).

2. Provide a path or link to a table that consists of at least two columns - one for the receptors and one for the    ligands. Each row of this table must correspond to a unique interaction, e.g. `http://tcm.zju.edu.cn/celltalkdb/download/processed_data/human_lr_pair.txt`

The databse will be stored in the adata object (`adata.uns['receptor-ligand]['database']`).

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Either a path/link to a table or the name of a LIANA resource
db_path = 'consensus'

ligand_column = 'ligand'
receptor_column = 'receptor'

_______

In [ ]:
rl.download_db(adata=adata,
               db_path=db_path,
               ligand_column=ligand_column,
               receptor_column=receptor_column,
               inplace=True,
               overwrite=False,
               report=("01_ligand_receptor_database.md", "01_ligand_receptor_table.png"))

--------------

## 5 - Compute interactions
Calculate a table of receptor-ligand interactions between given groups using the previously downloaded database. The resulting table is used to generate the plots in the next steps.
The resulting table contains:
- __receptor-/ ligand score__: Gene enriched (`> 0`) or depleted (`< 0`) for specific group.
- __receptor-/ ligand percent__: Percent of cells in a group expressing gene.
- __interaction score__: Receptor-ligand pair enriched (`> 0`) or depleted (`< 0`) between groups.

The interaction score is calculated by adding the receptor score to the ligand score with each score being multiplied by the respective cluster proportion to correct for cluster size:

$$\text{interaction_score} = score_{receptor} \times proportion_{\text{receptor cluster}} + score_{ligand} \times proportion_{\text{ligand cluster}} $$

The result table is stored in the adata object (`adata.uns['receptor-ligand']['interactions']`)

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Comput interactions
cluster_col = "clustering"  # Name of the cluster column in adata.obs.
normalize = None  # Normalize cell counts.
                  # Scales expression values to correct for different cluster sizes, using this as the reference cell count.
                  # If None, max cluster size will be used.
gene_col = None   # Column in adata.var that holds gene symbols/ ids. Set to None to use index
overwrite = False # Overwrite existing interaction table

# Data layer (advanced)
# decide which processing state of the data should be used for computation
# available layers can be seen above, e.g. "norm" to use normalized data
# it is recommended to use raw or normalized data for statistical testing
layer = "raw"

_______

In [ ]:
rl.calculate_interaction_table(adata=adata,
                               cluster_column=cluster_col,
                               gene_index=gene_col,
                               normalize=normalize,
                               inplace=True,
                               overwrite=overwrite,
                               layer=layer)

--------------

## 6 - Plotting
<hr style="border:2px solid black"> </hr>

### 6.1 - Violin
Show the distribution of interaction scores for all group combinations.

Scores `> 0` can be interpreted as receptor-ligand interactions enriched for group combination.  
Scores `< 0` can be interpreted as receptor-ligand interactions depleted for group combination.

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Violin plotting

violin_min_perc = 0  # Minimum percentage of cells in a cluster that express the respective gene.
violin_out_suffix = "violin"  # Suffix of output file
violin_figsize = (5, 30)  # Figure size

_______

In [ ]:
_ = rl.interaction_violin_plot(
    adata,
    min_perc=violin_min_perc,
    save=f"rl_{violin_out_suffix}.pdf",
    figsize=violin_figsize
)

__________

### 6.2 - Network
Show a network graph of number of interactions between groups.
Provided are two plots showing the interactions between groups:
1. Hairball plot: A simple network plot with the groups shown as nodes and the edges corresponding to the interactions between the groups.
2. Cyclone plot: A more stylized cicular plot with each segment represents one group and each edge also corresponding to the interactions between the groups. In addition to the network plot the cyclone plot can also show the top X receptors and ligands of each segment and is able to show the direction of the interactions (ligand -> receptor)

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
# Network plotting

net_min_perc = 0  # Minimum percentage of cells in a cluster that express the respective gene.
net_interaction_score = 0  # Interaction score must be above this threshold for the interaction to be counted in the graph.
net_interaction_perc = None  # Select interaction scores above or equal to the given percentile. Will overwrite parameter interaction_score.
net_out_suffix = "network"  # Suffix of output file
net_title = None  # Plot title
net_color_min = 0  # Min value for color range
net_color_max = None  # Max value for color range

# hairball specific parameters
net_restrict_to = []  # Only show given clusters provided in list.
net_show_count = True  # Show the interaction count
net_node_size = 10  # Set the size of the nodes.
net_label_size = 12  # Set the font size of the node labels.

# cyclone specific parameters
net_directional = False  # Show the direction of the interactions (ligand -> receptor)
net_sector_size_is_cluster_size = False  # Sector width is based on number of cells
net_show_genes = True  # Show the top receptor & ligand genes for each cluster
net_gene_amount = 5  # The amount of top receptor & ligand genes that are shown for each cluster

_______

In [ ]:
_ = rl.hairball(
    adata,
    min_perc=net_min_perc,
    interaction_score=net_interaction_score,
    interaction_perc=net_interaction_perc,
    save=f"rl_hairball_{net_out_suffix}.pdf",
    title=net_title,
    color_min=net_color_min,
    color_max=net_color_max,
    restrict_to=net_restrict_to,
    show_count=net_show_count,
    node_size = net_node_size,
    node_label_size = net_label_size
)

In [ ]:
_ = rl.cyclone(
    adata=adata,
    min_perc=net_min_perc,
    interaction_score=net_interaction_score,
    interaction_perc=net_interaction_perc,
    save=f"rl_cyclone_{net_out_suffix}.pdf",
    color_min=net_color_min,
    color_max=net_color_max,
    title=net_title,
    directional=net_directional,
    sector_size_is_cluster_size=net_sector_size_is_cluster_size,
    show_genes=net_show_genes,
    gene_amount=net_gene_amount,
    report = '02_cyclone_plot.png'
)

__________

### 6.3 - Receptor-ligand connections
Show a detailed view on receptor-ligand pairs and their strength between groups.

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
## Connection plot
con_restrict_to = None  # Restrict plot to given cluster names.
con_figsize = (5, 10)  # Figure size
con_out_suffix="connectionPlot"  # Suffix of output file
con_title = None  # Plot title
con_filter = "receptor_score > 0 & ligand_score > 0 & interaction_score > 0 & receptor_percent >= 30 & ligand_percent >= 30"  # Conditions to filter the interaction table
con_lw_multiplier = 2  # Linewidth multiplier
con_wspace = 0.4  # Width between plots
con_lig_whitelist = None  # List of ligand genes that should be exclusively shown.
con_rec_whitelist = None  # List of receptor genes that should be exclusively shown.
con_dot_size = (10, 100)  # Min and max size of the dots.
con_line_colors = "rainbow" #  Color scheme used to color the connection lines.
con_dot_colors = "flare"  # Color scheme used to color the dots.
con_xlabel_order = None  # A list of all xlabels. Will order the xlabels in the given order. None = alphabetically
con_alpha_range = None  # A min-max tuple (e.g. `(0, 10)`). Set the legend range of the lines alpha values. None = use data min and max.

_______

In [ ]:
_ = rl.connectionPlot(
    adata=adata,
    restrict_to=con_restrict_to,
    figsize=con_figsize,
    connection_alpha="interaction_score",
    save=f"rl_{con_out_suffix}.pdf",
    title=con_title,
    receptor_cluster_col="receptor_cluster",
    receptor_col="receptor_gene",
    receptor_hue="receptor_score",
    receptor_size="receptor_percent",
    receptor_genes=con_rec_whitelist,
    ligand_cluster_col="ligand_cluster",
    ligand_col="ligand_gene",
    ligand_hue="ligand_score",
    ligand_size="ligand_percent",
    ligand_genes=con_lig_whitelist,
    filter=con_filter,
    lw_multiplier=con_lw_multiplier,
    dot_size=con_dot_size,
    wspace=con_wspace,
    line_colors=con_line_colors,
    dot_colors=con_dot_colors,
    xlabel_order=con_xlabel_order,
    alpha_range=con_alpha_range,
    report = '03_connectionPlot.png'
)

__________

## 7 - Get interaction table

Filter and write receptor ligand interactions table from the anndata object to a .tsv file. 

<h1><center>⬐ Fill in input data here ⬎</center></h1>

In [ ]:
## Filter for interaction table
tab_min_perc = 0  # Minimum percent of cells in a cluster that express the ligand/ receptor gene.
tab_interaction_score = 0  # Filter receptor-ligand interactions below given score.
tab_interaction_perc = 90  # Filter receptor-ligand interactions below the given percentile. Overwrite `interaction_score`.
tab_out_suffix = "interaction_table"  # file name suffix
group_a = None  # List of cluster names that must be present in any given receptor-ligand interaction.
group_b = None  # List of cluster names that must be present in any given receptor-ligand interaction.

_______

In [ ]:
interaction_table = rl.get_interactions(
    adata,
    min_perc = tab_min_perc,
    interaction_score = tab_interaction_score,
    interaction_perc = tab_interaction_perc,
    group_a = group_a,
    group_b = group_b,
    save = f"rl_{tab_out_suffix}.tsv"
)

## 8 - Saving adata

In [ ]:
utils.adata.save_h5ad(adata, "anndata_0A1.h5ad")

In [ ]:
settings.close_logfile()